In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
import altair as alt
import string
import keras
import keras_hub
import keras_nlp
import re
import tensorflow as tf
import matplotlib.pyplot as plt

<div style="background-color:#e4f7f5; padding:20px; border-radius:5px;">
    <h1>🍕👒 Sentiment Analysis on Pizza Hut Reviews Dataset</h1>
    <p>I am using one of Kaggle's public datasets, the <a href="https://www.kaggle.com/datasets/abhi10699/pizza-hut-reviews">Pizza Hut Reviews Dataset</a>.</p>
    <blockquote>
        This dataset captures customer sentiments regarding Pizza Hut’s food quality, service, and overall experience. The data was collected through web scraping and consists of user-generated reviews along with associated ratings and metadata. I found this via Google Dataset Search.
    </blockquote>
    <h2>Data Overview</h2>
    <h3>About Data</h3>
    <blockquote>
        Data represents customer sentiments regarding Pizza Hut food and other services.
        Data was collected by web scraping: <a href="https://www.consumeraffairs.com/food/pizza-hut.html">Pizza Hut Food Reviews</a>
    </blockquote>
    <h3>Data Columns</h3>
    <ul>
        <li><code>id</code></li>
        <li><code>customer_review</code></li>
        <li><code>review_submitted_on_date</code></li>
        <li><code>overall_ratings</code></li>
        <li><code>upvotes_number</code></li>
    </ul>
    <h3>Possible Applications</h3>
    <ul>
        <li>Sentiment Analysis</li>
        <li>Can figure out the trend of reviews based on time</li>
    </ul>
</div>

## 🔎 Exploratory Data Analysis (EDA)

<div style="background: linear-gradient(to right, #d9edf7, #bce8f1); 
            color: #31708f; 
            padding: 15px; 
            border-radius: 5px;">
    
  <h3>Loading the Data into the Data Frame</h3>

  <h3>Sentiment Analysis</h3>
  <ul>
    <li>The <code>customer_review</code> column will be analyzed to determine sentiment (Positive, Neutral, or Negative).</li>
    <li>Sentiment classification will be performed using NLP techniques like VADER or TextBlob.</li>
    <li>Insights can be drawn to understand customer satisfaction and trends over time.</li>
  </ul>

  <h3>Train-Test Split</h3>
  <ul>
    <li>The dataset will be divided into training and test sets for further analysis and model development.</li>
    <li>A typical split ratio is <strong>80% for training</strong> and <strong>20% for testing</strong>.</li>
    <li>The split ensures robust performance evaluation of sentiment analysis models.</li>
  </ul>
</div>

In [ ]:
df = pd.read_csv("/kaggle/input/pizza-hut-reviews/pizza-hut-reviews.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

<div style="background: linear-gradient(to right, #d9edf7, #bce8f1); 
            color: #31708f; 
            padding: 15px; 
            border-radius: 5px; 
            border: 1px solid #bce8f1;
            display: inline-block;">
    <strong>Note:</strong> All columns have no missing values.
</div>

## 📊 VADER Sentiment Analysis
> VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.

📝 Memo: 
> nltk (Natural Language Toolkit) is a powerful Python library used for working with human language data. It provides tools for text processing, tokenization, part-of-speech tagging, and sentiment analysis, among other NLP tasks.

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER lexicon
nltk.download('vader_lexicon')

# Initialize VADER Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Function to get sentiment scores
def get_sentiment(text):
    scores = sia.polarity_scores(str(text))  # Convert to string in case of missing values
    if scores['compound'] >= 0.05:
        return "Positive"
    elif scores['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Apply function to classify sentiment
df["sentiment"] = df["review"].apply(get_sentiment)

# Show sample output
print(df[["review", "sentiment"]].head())

<section style="color: #333; background: linear-gradient(90deg, rgba(215,225,230,1) 0%, rgba(225,230,225,1) 50%, rgba(235,235,220,1) 100%); padding: 20px; font-size: 16px; border-radius: 8px; line-height: 1.5;">
  <p>
    It is obvious that this sentiment classifier is not 100% accurate. For example, a review starting with “I do not suggest you order from Pizza Hut, I l…” is simply labeled as “Positive,” which contradicts the customer's comment.
  </p>
  <p>
    According to the 
    <a href="https://hex.tech/templates/sentiment-analysis/vader-sentiment-analysis/#:~:text=This%20is%20it%2C%20you%20have,but%20it%20does%20the%20job." target="_blank" style="color: #333; text-decoration: underline;">
      How to Build: VADER Sentiment Analysis
    </a> guide, while VADER—a tool designed for capturing the sentiment of casual and informal language—achieves around 61% accuracy, that level of performance is often acceptable for many practical applications.
    <br><br>
    "This is it, you have now created a sentiment classifier with a few simple steps. VADER is a pretty handy tool for analyzing the sentiment of social media text as these platforms allow the use of casual and informal language. We were able to achieve around 61% accuracy which is not too good but it does the job."
  </p>
  <p>
    Given the inherent complexities and nuances of language that make sentiment analysis challenging for machines, I believe the achieved accuracy is reasonable; therefore, I will continue utilizing these results.
  </p>
</section>

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate

# Splitting dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=111)

# Features (X) and labels (y)
X_train = train_df["review"]
y_train = train_df["sentiment"]
X_test = test_df["review"]
y_test = test_df["sentiment"]

In [ ]:
class_dist = train_df['sentiment'].value_counts()
class_dist

<p style="color: white; background: #2A7B9B; background: linear-gradient(90deg, rgba(42, 123, 155, 1) 0%, rgba(87, 199, 133, 1) 50%, rgba(237, 221, 83, 1) 100%);; padding: 10px; font-size: 16px;">
I didn’t expect this response even though personally, I have rarely ordered food from Pizza Hut,  Given that it’s such a popular restaurant chain, it's surprising that people are this upset about the food and service.
I want to identify the words people use to express their reviews.
</p>

In [ ]:
from nltk.corpus import stopwords
from collections import Counter

# Download stopwords if necessary
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Combine all positive sentiment reviews into a single text string
positive_text = " ".join(df[df["sentiment"] == "Positive"]["review"].astype(str))

# Tokenize and clean the text (removing punctuation and converting to lowercase)
words = re.findall(r'\b\w+\b', positive_text.lower())

# Remove stopwords
filtered_words = [word for word in words if word not in stop_words]

# Count word frequencies
word_counts = Counter(filtered_words)

# Get the most common words
most_common_words = word_counts.most_common(20)  # Adjust the number as needed

print(most_common_words)

In [ ]:
# Download stopwords if necessary
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Combine all positive sentiment reviews into a single text string
negative_text = " ".join(df[df["sentiment"] == "Negative"]["review"].astype(str))

# Tokenize and clean the text (removing punctuation and converting to lowercase)
words = re.findall(r'\b\w+\b', negative_text.lower())

# Remove stopwords
filtered_words = [word for word in words if word not in stop_words]

# Count word frequencies
word_counts = Counter(filtered_words)

# Get the most common words
most_common_words = word_counts.most_common(20)  # Adjust the number as needed

print(most_common_words)

In [ ]:
from wordcloud import WordCloud

# List of words to exclude
exclude_words = {"pizza", "order", "hut", "ordered", "s"}

# Tokenize and clean the text
words = re.findall(r'\b\w+\b', positive_text.lower())

# Remove unwanted words
filtered_words = [word for word in words if word not in exclude_words]

# Recreate the text after filtering
filtered_text = " ".join(filtered_words)

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(filtered_text)

# Display the word cloud
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from wordcloud import WordCloud

# Filter only negative sentiment text entries
negative_text = " ".join(df[df["sentiment"] == "Negative"]["review"].astype(str))

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(negative_text)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(negative_text)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
class_dist.isnull().sum()

In [ ]:
review_length_df = train_df.copy()
review_length_df.loc[:, 'review_wordcount'] = review_length_df['review'].str.split().str.len()
review_length_df

📝 Memo
> This happens because every review in your column is unique. When you use value_counts(normalize=True), each unique review appears only once, so its normalized count is ( \frac{1}{\text{total number of reviews}} ). In your case, ( \frac{1}{605} \approx 0.001653 ), meaning your dataframe has roughly 605 unique reviews.

In [ ]:
df['review'].value_counts(normalize = True)

In [ ]:
# Calculate the average word count for negative sentiment reviews
neg_wc_avg = round(review_length_df[review_length_df["sentiment"] == "Negative"]["review_wordcount"].mean())

# Calculate the average word count for positive sentiment reviews
pos_wc_avg = round(review_length_df[review_length_df["sentiment"] == "Positive"]["review_wordcount"].mean())

# Print the results
print(neg_wc_avg)
print(pos_wc_avg)

In [ ]:
# Grouping data by sentiment and calculating the average review word count
avg_wc = review_length_df.groupby('sentiment', as_index=False)['review_wordcount'].mean()

# Creating the bar chart with different colors for each sentiment
plot_avg_wc = alt.Chart(avg_wc).mark_bar().encode(
    x=alt.X('sentiment:N', title='Sentiment'),
    y=alt.Y('review_wordcount:Q', title='Average Review Word Count'),
    color=alt.Color('sentiment:N', scale=alt.Scale(scheme='category20')) # Assigning colors
).properties(
    title='The Average Review Word Count per Sentiment',
    width=400 
)

# Display the chart
plot_avg_wc

## 🏷️ DummyClassifier

> DummyClassifier makes predictions that ignore the input features.
> 
> This classifier serves as a simple baseline to compare against other more complex classifiers.

In [ ]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train);
cv_results = cross_validate(dummy, X_train, y_train, cv=5, return_train_score=True)
dummy_scores = pd.DataFrame(cv_results)
dummy_scores

##### 💡 Cross-Validation Analysis

Observations by Microsoft Copilot

Fit Time & Score Time
- *Fit Time:* Ranges from `0.001174` to `0.001778` seconds, indicating a fast training process.
- *Score Time:* Even lower, ranging from `0.000967` to `0.001392` seconds, suggesting the model is efficient at making predictions.

Test Score & Train Score
- *Training Scores:* Consistent, ranging from `0.556701` to `0.558140`.
- *Test Scores:* Slightly lower in most cases, except for one (`0.562500`), suggesting minor variance.
- *Generalization:* The similarity between training and test scores indicates the model is not overfitting, but overall performance is relatively low.

Possible Next Steps
1. *Improve Model Performance*
   - Since your test scores hover around 0.56, consider feature engineering, hyperparameter tuning, or trying a more complex model.
2. *Check for Data Imbalance*
   - If your dataset is imbalanced, adjusting class weights or using resampling methods might help.
3. *Increase Training Data*
   - Using more data could improve model generalization.
4. *Hyperparameter Optimization*
   - Try techniques like Grid Search or Random Search to fine-tune model settings.

📌 <br>
I understand that DummyClassifier is a simple baseline model. 
<br>
I plan to use more advanced models later, as I want to leverage generative AI at this stage.

## 💎 Gemma 2 

<div style="background: linear-gradient(90deg, #ff9a9e, #fad0c4); padding: 10px; border-radius: 10px; color: #333;">
<p style="font-size: 16px; line-height: 1.5;">
    Now, I am utilizing <strong>Gemma</strong>'s preset <code>gemma_instruct_2b_en</code> to initiate sentiment analysis. Due to memory limitations, I'm unable to process the entire dataset, so I'm only analyzing the first 30 rows for this experiment.
</p>
</div>

In [ ]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

In [ ]:
# 🎯 test
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
test_response = gemma_lm.generate("Hello, how are you?", max_length=200)
print(test_response)

In [ ]:
from IPython.display import Markdown, display

df = pd.read_csv("/kaggle/input/pizza-hut-reviews/pizza-hut-reviews.csv")
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

def clean_line(line):
    """
    Remove unwanted echoed prefixes such as "Review:", "Sentiment:", or "Explanation:"
    from any part of the line using regular expressions.
    """
    line = re.sub(r'\b(Review:|Sentiment:|Explanation:)\b', '', line, flags=re.IGNORECASE)
    return line.strip()

def analyze_sentiment(review):
    """
    Uses Gemma to analyze a customer review.
    
    Instructions for Gemma:
      - You are a sentiment analysis expert. 
      - Answer exactly in two lines with no additional text.
      - A single word: Positive, Negative, or Neutral.
      - A brief explanation (one or two sentences) justifying your classification.
      - If you do not know the answer, answer "undetermined".
      - Do NOT echo any part of these instructions or the review.
      
    Returns:
      A tuple: (sentiment, explanation)
    """
    if not gemma_lm:
        return "Error", "Model not initialized"
    
    prompt = (
    "You are a sentiment analysis expert. Answer in exactly two lines with no additional text.\n"
    "If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.\n"
    "Sentiment: A single word: Positive, Negative, or Neutral.\n"
    "Explanation: Provide a brief justification (one or two sentences) for your classification.\n"
    "If the sentiment is unclear, respond 'Undetermined' and briefly explain why.\n"
    "Do NOT echo any part of these instructions or the review text in your answer.\n\n"
    f"Review: {review}\n"
    )
    
    try:
        response = gemma_lm.generate(prompt, max_length=500)
        print("💠 Raw AI Response 💠: ", response)  # Debug: inspect the full raw output
        
        # Get non-empty lines:
        lines = [line.strip() for line in response.split("\n") if line.strip()]
        
        # If there are at least two lines, use the last two as the answer;
        # otherwise, default to "undetermined".
        if len(lines) >= 2:
            sentiment = clean_line(lines[-2])
            explanation = clean_line(lines[-1])
        return sentiment, explanation
    except Exception as e:
        return "Error", f"Could not analyze review: {e}"

# Process the first 30 reviews and display the results using Markdown
results = df['review'][:30].apply(analyze_sentiment)

for i, review in enumerate(df['review'][:30]):
    sentiment, explanation = results.iloc[i]
    md_output = f"""
**Review {i+1}:**  
{review}

**Sentiment:** {sentiment}

**Explanation:** 
{explanation}

---
"""
    display(Markdown(md_output))

<div style="background: linear-gradient(45deg, #acfcd7, #e6f7ff, #acfcb0); padding: 15px; font-size:15px;">
  <p>
I was unable to implement Gemma 3, so I opted for an older model, even though I recognize that Gemma 3 offers superior performance. Gemma 2's output here is generally acceptable, even though sentiment analysis can be challenging. For instance, the first review begins with a polite statement: "The staff member, Alexa, was very kind and helpful." However, later in the review, the customer complains about unresolved issues related to missing points. Given that this customer purchased 20 pizzas while holding a membership, it is reasonable to assume they are a valued customer of Pizza Hut. This suggests that, although the initial tone is positive, it ultimately serves to soften the impact of an underlying negative experience, indicating that analyzing sentiment is not straightforward.
  </p>
</div>

<div style="background-color: #b5e38d; padding: 1rem; border-radius: 4px; line-height: 1.5;">
  <p style="margin: 0;">
    <strong>Below are the outputs for indices 30 through 59 (selected using <code>df['review'][30:60]</code>).</strong>
  </p>
  <p>
    All sentiments were categorized as <strong>Negative</strong>, and these assessments were accurate. I will not investigate all 605 entries further; however, I suspect that the VADER Sentiment Analysis results may not be entirely reliable. In the training data, the output was as follows:
  </p>
  <ul style="margin-top: 0; list-style: none; padding-left: 0;">
    <li><strong>Negative:</strong> 270</li>
    <li><strong>Positive:</strong> 200</li>
    <li><strong>Neutral:</strong> 14</li>
  </ul>
  <p>However, Gemma 2 identified the majority as negative. This is a small sample, but Gemma 2 demonstrated a better sentiment analysis result here.</p>
</div>

💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Ordered 3 pizzas from the County Line Rd location in Fayettvile. First and last time at any Pizza Hut: it was that bad. Cheese burnt onto crust, crust hard and straight as a board, zero sauce. It was so bad that even my 8-year-old said, "No thanks." Nice counter experience, but when we got home we were extremely disappointed. This was further on a Wednesday night, before 6:30 PM, so not a rush time. No good reason for the awful experience.
Sentiment: Negative
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: I recently ordered a large pizza on Jan, 7, 2022 at the Compton Ca location. The pizza was a 3 topping pan. There was literally no sauce, barely any cheese, and very little toppings. The pizza dough was not thoroughly cooked and very little flavor. The only thing good about the pizza, it was hot, and ready when I arrived. I really wish I would of taken the pizza back. We won't be ordering Pizza Hut anymore!
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the pizza's lack of flavor, freshness, and overall quality.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Ordered on Friday Dec. 31. Spent 50$ and got 2 medium pizzas, a 2L pop and some dips for my "missing" chicken wings. Called the phone number (online), they transferred me to another phone number (online), they transferred me to another phone number and I was put on hold for over 30 minutes.... Really sucked.
Sentiment: Negative
Explanation: The review contains negative language and expresses dissatisfaction with the service.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Been a while since I ordered, they changed the sauce. It stinks. No flavor. You're right next to Kraft and killing their mac and cheese recipe. You need to fire your research department and your CEOs. You don't fix what's not broke, now it's broke. No reason to buy from either of you again. Good decision!
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the changed sauce, implying it lacks flavor and is not a good substitute for the original recipe.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: The Pizza Hut in Passaic, New Jersey service are horrible. My order was missing an item and another item on my order was not what I ordered. They charged 50 cent for garlic sauce and didn't put it in the bag. I placed the order on online to avoid confusion. However, reading is a problem at this place as well. Just shut down the location.
Sentiment: Negative
Explanation: The review contains multiple issues with the order and service, indicating a negative sentiment.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: I ordered 2 pizza and breadsticks at 5:15 pm for delivery. At 6:13 I attempted to contact the store to see where my order was. I tried 4 times to reach the store but no answer. I had my husband call and she finally answered. He asked how much longer. She said order was ready but it would be another 1.5 hours because they only had 1 driver. I instructed my husband to file complaint and ask for 50% refund since the pizza has been sitting and probably dried out. They should refund driver tip which was included in order. If they
don't comply with this request, I will be taking my business elsewhere.

Sentiment: Negative
Explanation: The review expresses frustration and inconvenience due to the store's inability to communicate and fulfill orders on time.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: The Pizza Hut in Columbia Heights MN is the worst pizza place that I have ever had and the people there are so rude. I ordered my pizzas and it took them a little over 2 hours just to bring it to me. I live not even 2 miles away. When I called the lady was so rude to me I have never been treated like that from a pizza place. I'm very disappointed with everything about this place. If I could I would give this place a big double 0000.
Sentiment: Negative
Explanation: The review expresses dissatisfaction and frustration with the service and food quality.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: We placed an online order at 4:30pm for delivery, and we didn't get it until after 6pm. Not only was it cooled off, but it was only half our order. The delivery genius said he would let them know at the restaurant, to take it off our bill, but he didn't. I also called the store several times to follow-up, only to get the automated Pizza Hut voice-over. I wrote a complaint on the Pizza Hut website, which no one has responded (it's been more than 3 business days like they said to allow), and if you call customer service, you don't get anyone there, either. Just the same damn automated Pizza Hut voice-over that the Augusta, Maine restaurant had on repeat. It looks like this is the MO for Pizza Hut, just reading through the reviews on here from all over. I'm guessing the company is closing in 2022, because they don't seem to do anything to keep customers. They've certainly lost our business for good.
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the order fulfillment and customer service experience, implying a negative sentiment.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Placed an online ordered 6:25 p.m., was told order was received and would be ready approximately at 7:02. However that was not true! I stood and waited oh so patiently for 2 hrs! The order was still not ready for pick! The manager was very oh so rude while yelling in the store to the other associates to bring her whatever products she needed at the time! Oh and don’t let me leave out she refused to come to the front counter to speak with the customers upon our request. No one was removing the products from the oven as they came up therefore everything that was in the oven was very much so BEYOND BURNED! On top of that one of those burned products was MINE! 
Sentiment: Negative
Explanation: The review contains numerous instances of dissatisfaction and frustration, indicating a negative sentiment.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Order online but I did not get what I ordered. Call the store, the staff asked me to give the order back and renew the order. Suddenly she put me on hold for a year...(very long waiting) after realized, no one call or reply. I called back and put another order and asked to make sure not double order then she put me on hold for a month....(long waiting). I called back for the 3rd times, the customer service told me to forward the call to manager. I even asked, for how long the manager would pick up the call? She said soon. Unfortunately, no reply. No hope, the stuff just rude without any confirmation or anything, just put on hold for long time. They charged my card. I will never order again.
Sentiment: Negative
Explanation: The review contains a series of negative experiences with poor customer service, resulting in a negative sentiment.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Delay in delivering the orders and forgot one dish which was not punched in the system. I called the supervisor and waited nearly 10 minutes. He didn't show up. That's the second time I have same issue in the same restaurant.
Sentiment: Negative
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Few friends and I decided to order Pizza Hut, usually we do it online but every payment option was leading to an "error" of one for or another so I decided to call, and it led to customer service with people who didn't know anything. They didn't even know of any locations delivering in my area. They were trying to get me to fix the error on my phone when I just wanted to order pizza and when I asked if I could just order they said no, that's a different department so they transferred me. 
Sentiment: Negative.
Explanation: The review contains multiple issues with communication and customer service, indicating a negative sentiment.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Pathetic service. Worst experience I ever had. This is the first time I had ordered from Pizza Hut. Unfortunately it has been delayed by 1hour and 50 mins. In spite of that the pizzas were horrible. Awful taste. It tasted like a raw pizza without any cheese and toppings. I had paid extra for cheese but there was no cheese neither any veggies. It was empty. I never ever expected this from pizza hut. Which is known for the best pizzas across the world. I called up customer care but there also I did not get any response. Neither they have refund my money nor they entertained it seriously. I did a big mistake ordering from here.
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the quality of the pizzas, with the customer experiencing a poor dining experience due to long wait times and subpar food quality.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: I paid $24 for a Detroit style pizza. I was notified it was on the way (I live 15 minutes or less from the restaurant) and it took an hour to arrive. The pizza was cold, and the toppings were ridiculously sparse. I paid for pepperoni (there was none) and sausage (there were about 12 pieces) and mushroom (there were around 8 pieces). The pizza was almost bare. I tried to call the restaurant and was told I would be transferred and was on hold for over 5 minutes. I'm done. I will never order from Pizza Hut again. I can get 3 pizzas from Dominos at this price that are hot, fresh, and chock full of toppings. Pizza Hut, you can do better.
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the quality of the pizza, including cold pizza, sparse toppings, and long wait time.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Ordered a pizza. 1 hour later got a call from a neighbor saying they received my pizza. Called the HUT and All I Got Was Bad Attitude. They claimed it was properly delivered. Out 30 bucks and no dinner. The so-called manager had very bad attitude and no customer service experience. It Was Like Talking To A Scripted 12 year old. Never again...Reversing The Charges....Bye HUT.
Sentiment: Negative
Explanation: The review contains negative language and expresses dissatisfaction with the service received.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Ordered a delivery pizza and they left it on the sidewalk in the rain under a doorbell by a door without notifying me. I did not check contactless but they choose to do it that way. Do not prepay at this restaurant.
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the restaurant's contactless delivery process, which resulted in the pizza being left on the sidewalk in the rain.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: It was so embarassing coming staright to Pizza hut counter and talking to front end Vipin in GTB Nagar Delhi on 2nd Oct. At the counter as when I was checking the rate and details, he arrogantly handed over the menu to me and told to, "Check the menu on your own" and later asked me to sit at the dining table. I was really shocked with the unprofessional attitude. So later I have booked online and promised myself that I will never come to Pizza hut counter.
Sentiment: Negative
Explanation: The review expresses a negative sentiment towards the customer service experience at the Pizza hut counter. The customer feels disrespected and unimportant by the staff.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Their service has went to **!!! Their call line, nobody picks up and when they finally do they hang up or don't know ** so they forward your call. They charged me 4 times, no refund!!! 2 hrs later no pizza!! I am extremely disgusted in Pizza Hut and their cs and hospitality. Papa John's is my new providers. I advise everybody to change to papa johns.
Sentiment: Negative
Explanation: The review contains multiple complaints about service quality, order accuracy, and customer service.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: I ordered pizza from Pizza Hut and being a very old customer I expected a good taste but this wasn't the same taste that use to be there. It was disappointing. And moreover, the delivery was also late in spite giving the order so early almost 30 mins late.
Sentiment: Negative
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: I ordered pizza at this location for delivery and they forgot my sauces. I called to see if they could bring them to me and the manager said that they were not going to send my sauces with any driver. They wanted to give me a credit but I ordered the sauces and I told them that they should have not forgotten them. They said that people make mistakes and I agree but mistakes are made to be fixed.
Sentiment: Negative
Explanation: The manager's response indicates a lack of empathy and a disregard for the customer's needs.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Ordered Pizza online. Paid with card. Even left a tip on transaction. Guess what? Two hours later no pizza. will not support this company again. Local pizza shops are better. Do not write anything bad. Consumer Affairs does not like it.
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the order not being delivered on time and the lack of communication from the company.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: I used the online app to order one large pizza with 3 toppings. The toppings mushrooms, black olives, and banana peppers. This pizza was intended to feed a family with small children. I also ordered a side of bottle of pop and bread sticks. I also asked for curb side pick up so I didn’t have to get the children out of their seats to go in. Because of this I tipped 25 percent on my order. Big mistake!!! I got there. The app wasn’t working so I couldn’t check in. I tried calling but no one picked up. I kept trying to call. Eventually gave up. I got the kids all undone from their car seats. Went in. I noticed there was only one family in the lobby. All six employees were messing around not one of them working or cooking and the phone left off the hook. I got my pizza and bread sticks.
Sentiment: Negative
Explanation: The review contains multiple issues with the app, lack of customer service, and poor communication.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: We tried the new Detroit thick crust pizza. It's rectangular and we had the pepperoni with mushrooms. They pour extra sauce on top of the cheese and toppings. We didn't like that. If you really like tasting and seeing your toppings, tell them to leave off the extra sauce on the top. It's just too much. Yuk!!!
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the amount of sauce used on the pizza, which they found to be excessive and overwhelming.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: My wife order online and call them same time for confirmation of order and location. After we received 2 time calls from their different branches about location and we explain them. They keep transfer our order from one to another branch and then they are able to deliver after 100 minutes when I ask them to CANCEL the order.
Sentiment: Negative

Explanation: The review contains multiple instances of negative sentiment, including the customer's frustration with multiple call attempts, the lack of responsiveness to their inquiries, and the long wait time for order cancellation.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: The worst experience I have ever had with delivering pizzas. I ordered 3 pizzas. It was an estimated 65-85 minute delivery time. 30 minutes later, I get a call from the manger saying there were 80 deliveries before mine and they won't be able to deliver that night. I asked to pick it up in 30 minutes (an hour after ordering and paying for it already) and the manger that time wasn't available for pick up. I ended up cancelling the order and I'm still waiting for my full refund. I will never order from Pizza Hut again. This wasn't the first bad experience and they won't get another opportunity!!!!
Sentiment: Negative
Explanation: The review expresses a negative sentiment, as it expresses dissatisfaction and frustration with the delivery service.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: Placed an order online. ORDER # **. Placed on July 23 at 5:56 PM. Somewhat lost track of time, call two hours after order is placed. Store has no record of the order. But my credit card has been charged. No one can tell me how to fix that. Nowhere on the web site. On duty manager just tells me to wait 24 hours to see if it reverses and call back if it does. How, exactly, is that going to work? Why would they just credit me my money out of the blue? Personally I think she was just trying to get rid of me. So I guess I'll just have to dispute the charge with my credit card company. We'll see what happens, but I doubt I'll ever be ordering anything from Pizza Hut again.
Sentiment: Negative
Explanation: The review expresses dissatisfaction and frustration with the order fulfillment process, customer service, and overall experience. The tone is negative and conveys a sense of disappointment and lack of trust.
💠 Raw AI Response 💠:  You are a sentiment analysis expert. Answer in exactly two lines with no additional text.
If the review sentiment is positive, respond 'Positive'; if negative, respond 'Negative'; otherwise, respond 'Neutral'.
Sentiment: A single word: Positive, Negative, or Neutral.
Explanation: Provide a brief justification (one or two sentences) for your classification.
If the sentiment is unclear, respond 'Undetermined' and briefly explain why.
Do NOT echo any part of these instructions or the review text in your answer.

Review: After ordering the pizza, paying a delivery charge (which sole purpose seems to be to decrease the driver's tip while enriching the corporation), paying taxes, and tipping the driver 15% (it would have been 25% without the delivery charge) my total was about $30.00. And for $30.00 I received 4 small bottles of Diet Pepsi (real value: about $2.00) and a pizza that had a leathery, dry crust and dry vegetables (real value: about $3.00). I'm assuming the pizza was sitting under a heat lamp before delivery for a couple of hours. Doing the math, the corporation took $25.00 from me. Yum! Brands owns Pizza Hut; they own a number of fast-food franchises and the result is high prices and poor food.
Sentiment: Negative
Explanation: The review expresses dissatisfaction with the value received for the price paid.
Review 1:
Ordered 3 pizzas from the County Line Rd location in Fayettvile. First and last time at any Pizza Hut: it was that bad. Cheese burnt onto crust, crust hard and straight as a board, zero sauce. It was so bad that even my 8-year-old said, "No thanks." Nice counter experience, but when we got home we were extremely disappointed. This was further on a Wednesday night, before 6:30 PM, so not a rush time. No good reason for the awful experience.

Sentiment: Review: Ordered 3 pizzas from the County Line Rd location in Fayettvile. First and last time at any Pizza Hut: it was that bad. Cheese burnt onto crust, crust hard and straight as a board, zero sauce. It was so bad that even my 8-year-old said, "No thanks." Nice counter experience, but when we got home we were extremely disappointed. This was further on a Wednesday night, before 6:30 PM, so not a rush time. No good reason for the awful experience.

Explanation: Sentiment: Negative

Review 2:
I recently ordered a large pizza on Jan, 7, 2022 at the Compton Ca location. The pizza was a 3 topping pan. There was literally no sauce, barely any cheese, and very little toppings. The pizza dough was not thoroughly cooked and very little flavor. The only thing good about the pizza, it was hot, and ready when I arrived. I really wish I would of taken the pizza back. We won't be ordering Pizza Hut anymore!

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the pizza's lack of flavor, freshness, and overall quality.

Review 3:
Ordered on Friday Dec. 31. Spent 50$ and got 2 medium pizzas, a 2L pop and some dips for my "missing" chicken wings. Called the phone number (online), they transferred me to another phone number (online), they transferred me to another phone number and I was put on hold for over 30 minutes.... Really sucked.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains negative language and expresses dissatisfaction with the service.

Review 4:
Been a while since I ordered, they changed the sauce. It stinks. No flavor. You're right next to Kraft and killing their mac and cheese recipe. You need to fire your research department and your CEOs. You don't fix what's not broke, now it's broke. No reason to buy from either of you again. Good decision!

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the changed sauce, implying it lacks flavor and is not a good substitute for the original recipe.

Review 5:
The Pizza Hut in Passaic, New Jersey service are horrible. My order was missing an item and another item on my order was not what I ordered. They charged 50 cent for garlic sauce and didn't put it in the bag. I placed the order on online to avoid confusion. However, reading is a problem at this place as well. Just shut down the location.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains multiple issues with the order and service, indicating a negative sentiment.

Review 6:
I ordered 2 pizza and breadsticks at 5:15 pm for delivery. At 6:13 I attempted to contact the store to see where my order was. I tried 4 times to reach the store but no answer. I had my husband call and she finally answered. He asked how much longer. She said order was ready but it would be another 1.5 hours because they only had 1 driver. I instructed my husband to file complaint and ask for 50% refund since the pizza has been sitting and probably dried out. They should refund driver tip which was included in order. If they

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses frustration and inconvenience due to the store's inability to communicate and fulfill orders on time.

Review 7:
The Pizza Hut in Columbia Heights MN is the worst pizza place that I have ever had and the people there are so rude. I ordered my pizzas and it took them a little over 2 hours just to bring it to me. I live not even 2 miles away. When I called the lady was so rude to me I have never been treated like that from a pizza place. I'm very disappointed with everything about this place. If I could I would give this place a big double 0000.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction and frustration with the service and food quality.

Review 8:
We placed an online order at 4:30pm for delivery, and we didn't get it until after 6pm. Not only was it cooled off, but it was only half our order. The delivery genius said he would let them know at the restaurant, to take it off our bill, but he didn't. I also called the store several times to follow-up, only to get the automated Pizza Hut voice-over. I wrote a complaint on the Pizza Hut website, which no one has responded (it's been more than 3 business days like they said to allow), and if you call customer service, you don't get anyone there, either. Just the same damn automated Pizza Hut voice-over that the Augusta, Maine restaurant had on repeat. It looks like this is the MO for Pizza Hut, just reading through the reviews on here from all over. I'm guessing the company is closing in 2022, because they don't seem to do anything to keep customers. They've certainly lost our business for good.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the order fulfillment and customer service experience, implying a negative sentiment.

Review 9:
Placed an online ordered 6:25 p.m., was told order was received and would be ready approximately at 7:02. However that was not true! I stood and waited oh so patiently for 2 hrs! The order was still not ready for pick! The manager was very oh so rude while yelling in the store to the other associates to bring her whatever products she needed at the time! Oh and don’t let me leave out she refused to come to the front counter to speak with the customers upon our request. No one was removing the products from the oven as they came up therefore everything that was in the oven was very much so BEYOND BURNED! On top of that one of those burned products was MINE!

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains numerous instances of dissatisfaction and frustration, indicating a negative sentiment.

Review 10:
Order online but I did not get what I ordered. Call the store, the staff asked me to give the order back and renew the order. Suddenly she put me on hold for a year...(very long waiting) after realized, no one call or reply. I called back and put another order and asked to make sure not double order then she put me on hold for a month....(long waiting). I called back for the 3rd times, the customer service told me to forward the call to manager. I even asked, for how long the manager would pick up the call? She said soon. Unfortunately, no reply. No hope, the stuff just rude without any confirmation or anything, just put on hold for long time. They charged my card. I will never order again.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains a series of negative experiences with poor customer service, resulting in a negative sentiment.

Review 11:
Delay in delivering the orders and forgot one dish which was not punched in the system. I called the supervisor and waited nearly 10 minutes. He didn't show up. That's the second time I have same issue in the same restaurant.

Sentiment: Review: Delay in delivering the orders and forgot one dish which was not punched in the system. I called the supervisor and waited nearly 10 minutes. He didn't show up. That's the second time I have same issue in the same restaurant.

Explanation: Sentiment: Negative

Review 12:
Few friends and I decided to order Pizza Hut, usually we do it online but every payment option was leading to an "error" of one for or another so I decided to call, and it led to customer service with people who didn't know anything. They didn't even know of any locations delivering in my area. They were trying to get me to fix the error on my phone when I just wanted to order pizza and when I asked if I could just order they said no, that's a different department so they transferred me.

Sentiment: Sentiment: Negative.

Explanation: Explanation: The review contains multiple issues with communication and customer service, indicating a negative sentiment.

Review 13:
Pathetic service. Worst experience I ever had. This is the first time I had ordered from Pizza Hut. Unfortunately it has been delayed by 1hour and 50 mins. In spite of that the pizzas were horrible. Awful taste. It tasted like a raw pizza without any cheese and toppings. I had paid extra for cheese but there was no cheese neither any veggies. It was empty. I never ever expected this from pizza hut. Which is known for the best pizzas across the world. I called up customer care but there also I did not get any response. Neither they have refund my money nor they entertained it seriously. I did a big mistake ordering from here.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the quality of the pizzas, with the customer experiencing a poor dining experience due to long wait times and subpar food quality.

Review 14:
I paid $24 for a Detroit style pizza. I was notified it was on the way (I live 15 minutes or less from the restaurant) and it took an hour to arrive. The pizza was cold, and the toppings were ridiculously sparse. I paid for pepperoni (there was none) and sausage (there were about 12 pieces) and mushroom (there were around 8 pieces). The pizza was almost bare. I tried to call the restaurant and was told I would be transferred and was on hold for over 5 minutes. I'm done. I will never order from Pizza Hut again. I can get 3 pizzas from Dominos at this price that are hot, fresh, and chock full of toppings. Pizza Hut, you can do better.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the quality of the pizza, including cold pizza, sparse toppings, and long wait time.

Review 15:
Ordered a pizza. 1 hour later got a call from a neighbor saying they received my pizza. Called the HUT and All I Got Was Bad Attitude. They claimed it was properly delivered. Out 30 bucks and no dinner. The so-called manager had very bad attitude and no customer service experience. It Was Like Talking To A Scripted 12 year old. Never again...Reversing The Charges....Bye HUT.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains negative language and expresses dissatisfaction with the service received.

Review 16:
Ordered a delivery pizza and they left it on the sidewalk in the rain under a doorbell by a door without notifying me. I did not check contactless but they choose to do it that way. Do not prepay at this restaurant.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the restaurant's contactless delivery process, which resulted in the pizza being left on the sidewalk in the rain.

Review 17:
It was so embarassing coming staright to Pizza hut counter and talking to front end Vipin in GTB Nagar Delhi on 2nd Oct. At the counter as when I was checking the rate and details, he arrogantly handed over the menu to me and told to, "Check the menu on your own" and later asked me to sit at the dining table. I was really shocked with the unprofessional attitude. So later I have booked online and promised myself that I will never come to Pizza hut counter.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses a negative sentiment towards the customer service experience at the Pizza hut counter. The customer feels disrespected and unimportant by the staff.

Review 18:
Their service has went to **!!! Their call line, nobody picks up and when they finally do they hang up or don't know ** so they forward your call. They charged me 4 times, no refund!!! 2 hrs later no pizza!! I am extremely disgusted in Pizza Hut and their cs and hospitality. Papa John's is my new providers. I advise everybody to change to papa johns.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains multiple complaints about service quality, order accuracy, and customer service.

Review 19:
I ordered pizza from Pizza Hut and being a very old customer I expected a good taste but this wasn't the same taste that use to be there. It was disappointing. And moreover, the delivery was also late in spite giving the order so early almost 30 mins late.

Sentiment: Review: I ordered pizza from Pizza Hut and being a very old customer I expected a good taste but this wasn't the same taste that use to be there. It was disappointing. And moreover, the delivery was also late in spite giving the order so early almost 30 mins late.

Explanation: Sentiment: Negative

Review 20:
I ordered pizza at this location for delivery and they forgot my sauces. I called to see if they could bring them to me and the manager said that they were not going to send my sauces with any driver. They wanted to give me a credit but I ordered the sauces and I told them that they should have not forgotten them. They said that people make mistakes and I agree but mistakes are made to be fixed.

Sentiment: Sentiment: Negative

Explanation: Explanation: The manager's response indicates a lack of empathy and a disregard for the customer's needs.

Review 21:
Ordered Pizza online. Paid with card. Even left a tip on transaction. Guess what? Two hours later no pizza. will not support this company again. Local pizza shops are better. Do not write anything bad. Consumer Affairs does not like it.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the order not being delivered on time and the lack of communication from the company.

Review 22:
I used the online app to order one large pizza with 3 toppings. The toppings mushrooms, black olives, and banana peppers. This pizza was intended to feed a family with small children. I also ordered a side of bottle of pop and bread sticks. I also asked for curb side pick up so I didn’t have to get the children out of their seats to go in. Because of this I tipped 25 percent on my order. Big mistake!!! I got there. The app wasn’t working so I couldn’t check in. I tried calling but no one picked up. I kept trying to call. Eventually gave up. I got the kids all undone from their car seats. Went in. I noticed there was only one family in the lobby. All six employees were messing around not one of them working or cooking and the phone left off the hook. I got my pizza and bread sticks.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains multiple issues with the app, lack of customer service, and poor communication.

Review 23:
We tried the new Detroit thick crust pizza. It's rectangular and we had the pepperoni with mushrooms. They pour extra sauce on top of the cheese and toppings. We didn't like that. If you really like tasting and seeing your toppings, tell them to leave off the extra sauce on the top. It's just too much. Yuk!!!

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the amount of sauce used on the pizza, which they found to be excessive and overwhelming.

Review 24:
My wife order online and call them same time for confirmation of order and location. After we received 2 time calls from their different branches about location and we explain them. They keep transfer our order from one to another branch and then they are able to deliver after 100 minutes when I ask them to CANCEL the order.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review contains multiple instances of negative sentiment, including the customer's frustration with multiple call attempts, the lack of responsiveness to their inquiries, and the long wait time for order cancellation.

Review 25:
The worst experience I have ever had with delivering pizzas. I ordered 3 pizzas. It was an estimated 65-85 minute delivery time. 30 minutes later, I get a call from the manger saying there were 80 deliveries before mine and they won't be able to deliver that night. I asked to pick it up in 30 minutes (an hour after ordering and paying for it already) and the manger that time wasn't available for pick up. I ended up cancelling the order and I'm still waiting for my full refund. I will never order from Pizza Hut again. This wasn't the first bad experience and they won't get another opportunity!!!!

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses a negative sentiment, as it expresses dissatisfaction and frustration with the delivery service.

Review 26:
My husband and I decided as of his testing clear of COVID, we’d treat ourselves to a pizza. We chose Pizza Hut because it was close. Ordered online for curbside pickup and we got an email with instructions. We arrived and there were no designated anything for curbside so as directed in the email, we called to alert them we were there.

Sentiment: Review: My husband and I decided as of his testing clear of COVID, we’d treat ourselves to a pizza. We chose Pizza Hut because it was close. Ordered online for curbside pickup and we got an email with instructions. We arrived and there were no designated anything for curbside so as directed in the email, we called to alert them we were there.

Explanation: The person on the phone was very rude and dismissive. She said we needed to go inside to order and that we were inconveniencing her. We were very disappointed by the experience and will not be returning to Pizza Hut.

Review 27:
For the first time in several years I ordered a pizza online to pick up at Pizza Hut. I should have read these reviews first. My pizza was ready a half hour after it was supposed to be brought to my car. It was overcooked and so salty as to be inedible. So unhealthy. I looked it up and found that Pizza Hut in America uses more than double the salt that it uses in other countries. Shame on them. Do not feed this to your children or eat it yourself.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the quality of the pizza, citing overcooking and saltiness.

Review 28:
I ordered over an hour ago for a 45 minute deliver and I called the store and they have no record of the order. I paid with a credit card. Not the first time this has happened. This is going to be the last time I order from Pizza Hut. Order from anywhere else until they get their act together. Horrible.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the order not being delivered on time and the lack of communication from the store.

Review 29:
Placed an order online. ORDER # **. Placed on July 23 at 5:56 PM. Somewhat lost track of time, call two hours after order is placed. Store has no record of the order. But my credit card has been charged. No one can tell me how to fix that. Nowhere on the web site. On duty manager just tells me to wait 24 hours to see if it reverses and call back if it does. How, exactly, is that going to work? Why would they just credit me my money out of the blue? Personally I think she was just trying to get rid of me. So I guess I'll just have to dispute the charge with my credit card company. We'll see what happens, but I doubt I'll ever be ordering anything from Pizza Hut again.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction and frustration with the order fulfillment process, customer service, and overall experience. The tone is negative and conveys a sense of disappointment and lack of trust.

Review 30:
After ordering the pizza, paying a delivery charge (which sole purpose seems to be to decrease the driver's tip while enriching the corporation), paying taxes, and tipping the driver 15% (it would have been 25% without the delivery charge) my total was about 
30.00 I received 4 small bottles of Diet Pepsi (real value: about 
3.00). I'm assuming the pizza was sitting under a heat lamp before delivery for a couple of hours. Doing the math, the corporation took $25.00 from me. Yum! Brands owns Pizza Hut; they own a number of fast-food franchises and the result is high prices and poor food.

Sentiment: Sentiment: Negative

Explanation: Explanation: The review expresses dissatisfaction with the value received for the price paid.

<p style="background: linear-gradient(90deg, #8aeb94, #8ae1eb); padding: 20px;">
I was very impressed by the explanation of Gemma 2 as addressed here.
</p>

<p style="background: linear-gradient(90deg, #fceabb, #f8b500); padding: 20px;">
  I would employ an AI model to uncover the underlying factors driving the frequent recurrence of negative reviews.
  <br>
  Due to memory constraints, I'm limiting the analysis to the last 10 rows.
</p>

In [ ]:
# Load the GemmaCausalLM model using its preset.
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

def analyze_reviews(reviews, max_length=200):
    """
    Uses the GemmaCausalLM model to generate an analysis for each review.
    
    Args:
        reviews (list of str): List of review texts.
        max_length (int): Maximum token length for the generated analysis.
    
    Returns:
        list of str: Generated analysis for each review.
    """
    analysis_results = []
    for i, review in enumerate(reviews, start=1):
        prompt = (
            f"Review: '{review}'\n"
            "Please analyze the customer's concerns expressed in this review."
        )
        analysis = gemma_lm.generate(prompt, max_length=max_length)
        analysis_results.append(f"Analysis {i}:\n{analysis}\n")
    return analysis_results

reviews = df['review'].tail(10).tolist()

analyses = analyze_reviews(reviews, max_length=200)

for analysis in analyses:
    print(analysis)

<div style="background: linear-gradient(45deg, #acfcd7, #e6f7ff, #acfcb0); padding: 15px; font-size:15px;">
<p>
Overall, the results are impressive. Looking ahead, I believe AI agents will continue to evolve, significantly enhancing customer satisfaction by delivering prompt, accurate, and considerate service. Long wait times for customer support can be incredibly frustrating, and addressing negative complaints is often challenging. By leveraging AI to streamline responses and manage difficult interactions more effectively, businesses may greatly improve the customer experience. I hope AI transforms our world into a better place for everyone.

In summary, this analysis has provided valuable insights. It highlights important trends and emphasizes the potential impact of AI-driven solutions in customer service. Moving forward, continued exploration and refinement in this area will be crucial for enhancing user experience and satisfaction.
</p>
</div>